In [12]:
import os
import sys
import gc
import time
import random
import cv2
import glob
import requests
import json
import math
import re
import scipy

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from functools import partial
from collections import Counter
from collections import defaultdict
from PIL import Image
from multiprocessing import cpu_count
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler

from fastai import *
from fastai.vision import *
from fastai.callbacks import *

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score

In [3]:
pd.options.display.max_columns = None

In [4]:
pre_df = pd.read_csv(os.path.join('..', 'input', 'retinopathy-train-2015', 'trainLabels.csv'))

In [5]:
image_name_list = os.listdir(os.path.join('..', 'input', 'retinopathy-train-2015', 'rescaled_train_896'))
image_name_path_list = [os.path.join('..', 'input', 'retinopathy-train-2015', 'rescaled_train_896', f'{image}') for image in image_name_list]
pre_df['path'] = image_name_path_list

In [6]:
base_image_dir = os.path.join('..', 'input')
train_dir = os.path.join(base_image_dir, 'train_images')
train_df = pd.read_csv(os.path.join(base_image_dir, 'train.csv'))
train_df['path'] = train_df['id_code'].map(lambda x: os.path.join(train_dir, '{}.png'.format(x)))

In [7]:
base_image_dir = os.path.join('..', 'input')
test_dir = os.path.join(base_image_dir, 'test_images')
test_df = pd.read_csv(os.path.join(base_image_dir, 'test.csv'))
test_df['path'] = test_df['id_code'].map(lambda x: os.path.join(test_dir, '{}.png'.format(x)))

In [8]:
def get_image_size_list(image_name_path_list):
    width = []
    height = []
    
    for image_name in tqdm(image_name_path_list):
        image = cv2.imread(image_name)
        w, h, _ = image.shape
        width.append(w)
        height.append(h)
    
    return width, height

In [9]:
def get_image_brightness_list(image_name_path_list):
    image_brightness_list = []
    for image_name in tqdm(image_name_path_list):
        image = cv2.imread(image_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (224, 224))
        image_brightness_list.append(np.mean(image))
    
    return image_brightness_list

In [10]:
pre_brightness = get_image_brightness_list(pre_df['path'])
train_brightness = get_image_brightness_list(train_df['path'])
test_brightness = get_image_brightness_list(test_df['path'])

100%|██████████| 1928/1928 [00:43<00:00, 44.14it/s]


In [11]:
pre_df['brightness'] = pre_brightness
train_df['brightness'] = train_brightness
test_df['brightness'] = test_brightness

del pre_brightness, train_brightness, test_brightness
gc.collect()

49